# 📝 Exercise M1.04

The goal of this exercise is to evaluate the impact of using an arbitrary
integer encoding for categorical variables along with a linear classification
model such as Logistic Regression.

To do so, let's try to use `OrdinalEncoder` to preprocess the categorical
variables. This preprocessor is assembled in a pipeline with
`LogisticRegression`. The generalization performance of the pipeline can be
evaluated by cross-validation and then compared to the score obtained when
using `OneHotEncoder` or to some other baseline score.

First, we load the dataset.

In [1]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

In [2]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

In the previous notebook, we used `sklearn.compose.make_column_selector` to
automatically select columns with a specific data type (also called `dtype`).
Here, we use this selector to get only the columns containing strings (column
with `object` dtype) that correspond to categorical features in our dataset.

In [3]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
data_categorical = data[categorical_columns]

Define a scikit-learn pipeline composed of an `OrdinalEncoder` and a
`LogisticRegression` classifier.

Because `OrdinalEncoder` can raise errors if it sees an unknown category at
prediction time, you can set the `handle_unknown="use_encoded_value"` and
`unknown_value` parameters. You can refer to the [scikit-learn
documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)
for more details regarding these parameters.

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

model = make_pipeline(encoder, LogisticRegression(max_iter=500))

cv_results = cross_validate(model, data_categorical, target, cv=5)

mean_score = cv_results["test_score"].mean()
std_score = cv_results["test_score"].std()

print(f"Ordinal Encoding Performance:")
print(f"Mean Accuracy: {mean_score:.3f} ± {std_score:.3f}")

Ordinal Encoding Performance:
Mean Accuracy: 0.755 ± 0.002



Be aware that if an error happened during the cross-validation, cross_validate would raise a warning and return NaN (Not a Number) as scores. To make it raise a standard Python exception with a traceback, you can pass the error_score="raise" argument in the call to cross_validate. An exception would be raised instead of a warning at the first encountered problem and cross_validate would stop right away instead of returning NaN values. This is particularly handy when developing complex machine learning pipelines

In [6]:
from sklearn.model_selection import cross_validate


cv_results = cross_validate(model, data_categorical, target, cv=5, error_score="raise")

scores = cv_results["test_score"]
print(f"The mean accuracy is: {scores.mean():.3f} +/- {scores.std():.3f}")

The mean accuracy is: 0.755 +/- 0.002


Now, we would like to compare the generalization performance of our previous
model with a new model where instead of using an `OrdinalEncoder`, we use a
`OneHotEncoder`. Repeat the model evaluation using cross-validation. Compare
the score of both models and conclude on the impact of choosing a specific
encoding strategy when using a linear model.

In [ ]:
from sklearn.dummy import DummyClassifier

baseline_model = DummyClassifier(strategy="most_frequent")

cv_results_baseline = cross_validate(
    baseline_model, data_categorical, target, cv=5
)

baseline_scores = cv_results_baseline["test_score"]
print(f"Baseline accuracy (most frequent class): {baseline_scores.mean():.3f} ± {baseline_scores.std():.3f}")

Baseline accuracy (most frequent class): 0.761 ± 0.000
